<a href="https://colab.research.google.com/github/akamalas5/Capstone/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraires

In [ ]:
import requests
from collections import defaultdict
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from tqdm.notebook import tqdm
import numpy as np
import ast
import csv

# Wikifier
import urllib
from string import punctuation
import nltk
import json
import itertools

import numpy as np
from sklearn.model_selection import train_test_split


## Create Train/Validation/test data sets from the input Mapping File

The relatioships are manually annotated with SME Knowledge

In [ ]:
input_file = "../data/wiki_model_input_trial_10K.csv"

output_dir = "../data/"
train_data = "wiki_train_df.txt"
test_data = "wiki_test_df.txt"

output_dir+train_data
output_dir+test_data

wiki_train_df = pd.read_csv(input_file)
wiki_train_df['data'] = wiki_train_df['data'].apply(ast.literal_eval) 

train_df, test_df = train_test_split(wiki_train_df, test_size=0.2, random_state=42, shuffle=True)
print(len(wiki_train_df), len(train_df), len(test_df))

train_df['data'].to_csv(output_dir+train_data,
                             quoting=csv.QUOTE_NONE, sep='\t', header = False, index = False)
test_df['data'].to_csv(output_dir+test_data,
                             quoting=csv.QUOTE_NONE, sep='\t', header = False, index = False)


10218 8174 2044


Rel2ID - Relationship to ID - Assing id to all the custom reltionships and generate rel2id file which is required by openNRE file

In [ ]:
import json
output_dir = "../data/"
out_file = 'wiki_model_rel2id.json'

brake_train_rel2id = {
    'is part of':0,
    'is type of':1, 
    'Relation':2, 
    'is related with':3, 
    'uses':4,
    'interacts with':5, 
    'is parameter of':6, 
    'indirectly related with':7,
    'consists of':8, 
    'has type':9, 
    'has parameter':10
}
f_rel2id_json = output_dir+out_file
with open(f_rel2id_json, "w") as fp:
    json.dump(brake_train_rel2id , fp) 

In [ ]:
 # Opening JSON file
f = open(f_rel2id_json)
 
# returns JSON object as
# a dictionary
data = json.load(f)
data

{'Relation': 2,
 'consists of': 8,
 'has parameter': 10,
 'has type': 9,
 'indirectly related with': 7,
 'interacts with': 5,
 'is parameter of': 6,
 'is part of': 0,
 'is related with': 3,
 'is type of': 1,
 'uses': 4}

Load OpenNRE
Prerequisite - Install OpenNRE follwing the instruciton provided in the Readme

In [ ]:
import os
import transformers
import opennre
import spacy

In [ ]:
# Project setup 
# import os
# os.chdir('/content/drive/My Drive/myEnv')
# import transformers
# os.chdir('/content/drive/My Drive/myEnv/OpenNRE')
# import opennre
# import spacy

In [ ]:
import torch
import numpy as np
import json
import opennre
from opennre import encoder, model, framework
import sys
import os
import argparse
import logging
import random

Set the consistent Seed value for all libraries

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(42)

Define the Entity Masked BERT model 

In [ ]:
def define_sentence_encoder(pooler = 'entity', max_length=128,
                            pretrain_path = 'bert-base-uncased',
                            mask_entity = 'mask_entity'):
  if pooler == 'entity':
    sentence_encoder = opennre.encoder.BERTEntityEncoder(
        max_length=max_length, 
        pretrain_path= pretrain_path,
        mask_entity=mask_entity
    )
  elif pooler == 'cls':
    sentence_encoder = opennre.encoder.BERTEncoder(
        max_length=max_length, 
        pretrain_path=pretrain_path,
        mask_entity=mask_entity
    )
  else:
    raise NotImplementedError

  return sentence_encoder
sentence_encoder = define_sentence_encoder()

2021-12-16 21:15:51,134 - root - INFO - Loading BERT pre-trained checkpoint.


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Split the Train data - Train, Val and Test

In [ ]:
train_file = "../data/wiki_train_df.txt"
val_file = "../data/wiki_test_df.txt"
test_file = "../data/wiki_test_df.txt"

rel2id_file = f_rel2id_json

The openNRE Model - Instantiation with Sentence Encoder

In [ ]:
rel2id = json.load(open(rel2id_file))
my_model = opennre.model.SoftmaxNN(sentence_encoder, len(rel2id), rel2id)

Train the model - With BERT Entity masked encoder

In [ ]:
# Point the ckpt value to the location in your openNRE installation OpenNRE/pretrain/bert-base-uncased/vocab.txt
vacab_file = '/OpenNRE/pretrain/bert-base-uncased/vocab.txt'

# Define the whole training framework
framework = opennre.framework.SentenceRE(
    train_path=train_file,
    val_path=val_file,
    test_path=test_file,
    model=my_model,
    ckpt=vacab_file,
    batch_size=64,
    max_epoch=3,
    lr=2e-5,
    opt='adamw'
)

2021-12-14 02:10:42,326 - root - INFO - Loaded sentence RE dataset /content/drive/My Drive/Capstone/data/train_data/wiki_train_df.txt with 8174 lines and 11 relations.
2021-12-14 02:10:42,505 - root - INFO - Loaded sentence RE dataset /content/drive/My Drive/Capstone/data/train_data/wiki_test_df.txt with 2044 lines and 11 relations.
2021-12-14 02:10:42,666 - root - INFO - Loaded sentence RE dataset /content/drive/My Drive/Capstone/data/train_data/wiki_test_df.txt with 2044 lines and 11 relations.


In [ ]:
#Trial  - End iteration flag is set to false in execption for debuggins
# File "/content/drive/My Drive/myEnv/torch/utils/data/_utils/worker.py", line 185,

In [ ]:
framework.train_model('micro_f1')

2021-12-14 02:10:50,999 - root - INFO - === Epoch 0 train ===
100%|██████████| 128/128 [01:33<00:00,  1.36it/s, acc=0.311, loss=1.96]
2021-12-14 02:12:24,909 - root - INFO - === Epoch 0 val ===
100%|██████████| 32/32 [00:08<00:00,  3.79it/s, acc=0.582]
2021-12-14 02:12:33,371 - root - INFO - Evaluation result: {'acc': 0.5821917808219178, 'micro_p': 0.5821917808219178, 'micro_r': 0.5821917808219178, 'micro_f1': 0.5821917808219178}.
2021-12-14 02:12:33,373 - root - INFO - Metric micro_f1 current / best: 0.5821917808219178 / 0
2021-12-14 02:12:33,375 - root - INFO - Best ckpt and saved.
2021-12-14 02:12:40,724 - root - INFO - === Epoch 1 train ===
100%|██████████| 128/128 [01:31<00:00,  1.39it/s, acc=0.714, loss=0.93]
2021-12-14 02:14:12,702 - root - INFO - === Epoch 1 val ===
100%|██████████| 32/32 [00:08<00:00,  3.79it/s, acc=0.818]
2021-12-14 02:14:21,146 - root - INFO - Evaluation result: {'acc': 0.8175146771037182, 'micro_p': 0.8175146771037182, 'micro_r': 0.8175146771037182, 'micro_

### Model Validation

In [ ]:
# Test
ckpt=vacab_file

framework.load_state_dict(torch.load(ckpt)['state_dict'])
result = framework.eval_model(framework.test_loader)

# Print the result
logging.info('Test set results:')
logging.info('Accuracy: {}'.format(result['acc']))
logging.info('Micro precision: {}'.format(result['micro_p']))
logging.info('Micro recall: {}'.format(result['micro_r']))
logging.info('Micro F1: {}'.format(result['micro_f1']))

100%|██████████| 32/32 [00:08<00:00,  3.78it/s, acc=0.89]
2021-12-14 02:16:13,899 - root - INFO - Evaluation result: {'acc': 0.8904109589041096, 'micro_p': 0.8904109589041096, 'micro_r': 0.8904109589041096, 'micro_f1': 0.8904109589041096}.
2021-12-14 02:16:13,900 - root - INFO - Test set results:
2021-12-14 02:16:13,901 - root - INFO - Accuracy: 0.8904109589041096
2021-12-14 02:16:13,904 - root - INFO - Micro precision: 0.8904109589041096
2021-12-14 02:16:13,905 - root - INFO - Micro recall: 0.8904109589041096
2021-12-14 02:16:13,906 - root - INFO - Micro F1: 0.8904109589041096


In [ ]:
data = my_model.infer(
    {'token': ['Some', 'vehicles', 'use', 'a', 'combination', 'of', 'braking', 'mechanisms'], 'h': {'pos': [1, 2]}, 't': {'pos': [6, 7]}}
)
print(data)

('indirectly related with', 0.47297462821006775)


## The the trained model file is avilable in the following location:
https://drive.google.com/drive/folders/1vGy_Us0rxn_JY6xw6v4NMHX6Z1HjHXtu?usp=sharing

In [ ]:
import pickle
filename = '/data/finalized_model.pkl'
#pickle.dump(my_model, open(filename, 'wb'))

In [ ]:
ls *.pkl

finalized_model.pkl


In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
data = loaded_model.infer(
    {'text': 'brake is a mechanical system, it is related to chassis', 'h': {'pos': [11, 21]}, 't': {'pos': [47, 54]}}
)
print(data)

('has type', 0.4834158420562744)


In [ ]:
data = loaded_model.infer(
    {'token': ['Some', 'vehicles', 'use', 'a', 'combination', 'of', 'braking', 'mechanisms'], 'h': {'pos': [1, 2]}, 't': {'pos': [6, 7]}}
)
print(data)

('indirectly related with', 0.47297462821006775)


Comparision of Results with openNRE Ready Models

In [ ]:
import itertools
 
relation_threshold = 0.5
# rel_model =  'wiki80_cnn_softmax'
# rel_model = 'wiki80_bert_softmax'
rel_model = 'wiki80_bertentity_softmax'
# Not returning many of the relations
# rel_model = 'tacred_bert_softmax'
# # the below one needs data set and retrain - licenced product
# rel_model = 'tacred_bertentity_softmax'

relation_model = opennre.get_model(rel_model)

2021-12-17 08:27:18,344 - root - INFO - Loading BERT pre-trained checkpoint.


In [ ]:
data = relation_model.infer(
    {'text': 'brake is a mechanical system, it is related to chassis', 'h': {'pos': [11, 21]}, 't': {'pos': [47, 54]}}
)
print(data)

('said to be the same as', 0.41218921542167664)


In [ ]:
data = relation_model.infer(
    {'token': ['Some', 'vehicles', 'use', 'a', 'combination', 'of', 'braking', 'mechanisms'], 'h': {'pos': [1, 2]}, 't': {'pos': [6, 7]}}
)
print(data)

('has part', 0.3066348731517792)




```
# This is formatted as code
```

Predict Links Using the Trained Model

In [ ]:
def remove_relation_key(input_dict):
  input_dict.pop('relation', None)

#input_file = '/content/drive/MyDrive/Capstone/data/train_data/wiki_mapping/dec_07_mapping_data_wiki_text_0.8.csv'
input_file = '../data/dec_07_mapping_data_req_text_0.8.csv'

input_df = pd.read_csv(input_file)
# Coneert data to dictionary
input_df['data'] = input_df['data'].apply(ast.literal_eval) 
# Remove the relation key - The actual relation should be populated with Predicted value
input_df.data.apply(remove_relation_key)


0        None
1        None
2        None
3        None
4        None
         ... 
72249    None
72250    None
72251    None
72252    None
72253    None
Name: data, Length: 72254, dtype: object

Generate the Output File - Prediction and Score for each of the Sentence from the Entire Brake Corpus ~60K sentenses

In [ ]:
df = input_df
preditions = []
for row in tqdm(df.data):
  result = loaded_model.infer(row)
  preditions.append(
      { 'relation' : result[0], 'score' : result[1]}
  )
pred_df = pd.DataFrame(preditions)
extracted_col1 = pred_df["relation"]
extracted_col2 = pred_df["score"]

#df = df.join(extracted_col)
df.insert(1,"relation", extracted_col1)
df.insert(1, "score", extracted_col2)


# df['score'] = pred_df['score']

df.to_csv("../data/with_train_test_mapping_data_req_text_0.8_results.csv")

  0%|          | 0/72254 [00:00<?, ?it/s]

In [ ]:
df.relation.value_counts()

indirectly related with    15685
is part of                 13569
consists of                12680
interacts with              8747
is related with             8620
is parameter of             5644
has parameter               4665
is type of                  1680
has type                     964
Name: relation, dtype: int64

In [ ]:
# With Model trainied with Wiki data without split
# indirectly related with    15211
# is part of                 12424
# consists of                11491
# is related with             9302
# interacts with              9142
# is parameter of             6634
# has parameter               5156
# is type of                  1756
# has type                    1138
# Name: relation, dtype: int64